In [1]:
!pip install langchain -q
!pip install pypdf2 -q
!pip install huggingface_hub -q
!pip install sentence_transformers -q
!pip install google-cloud-aiplatform  google-api-python-client vertexai transformers chromadb google-auth-oauthlib -q
!pip install pypdf -q

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import os
pdf_folder = '/content/drive/My Drive/Earning_reports/Concalls/ADANIENT'
os.chdir(pdf_folder)

In [12]:
os.listdir()

['Adani Enterprises_aug23.pdf',
 'Adani Enterprises_may23.pdf',
 'Adani Enterprises_Feb23.pdf',
 'Adani Enterprises_nov22.pdf',
 'Adani Enterprises_aug22.pdf',
 'Adani Enterprises_nov21.pdf',
 'Adani Enterprises_aug21.pdf',
 'Adani Enterprises_mar21.pdf',
 'Adani Enterprises_oct16.pdf',
 'Adani Enterprises_aug16.pdf',
 'Adani Enterprises_jan16.pdf',
 '21merged.pdf',
 '23merged.pdf',
 'key.json',
 'key (1).json',
 'vector_db']

In [13]:
pdf_files = [file for file in os.listdir() if file.endswith('.pdf')]

In [14]:
pdf_files[0]

'Adani Enterprises_aug23.pdf'

In [15]:
from langchain.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader(pdf_files[0])
docs = pdf_loader.load()

In [16]:
pages = docs[0]
print(len(docs))
print(pages.page_content[0:500])

print(pages.metadata)

12
        
 
Adani Enterprises Limited   
“Adani Corporate House”, 
Shantigram, Near Vaishno Devi Circle,  
S. G. Highway, Khodiyar 
Ahmedabad 382 421 
Gujarat, India Tel  + 91 79 2656 5555  
Fax + 91 79 2555 5500 
investor.ael@adani.com  
www.adanienterprises.com 
 
CIN: L51100GJ1993PLC019067   
  
Registered Office : “Adani Corporate House”, Shantigram, Near Vaishno Devi Circle, S. G. Highwa y, Khodiyar, Ahmedabad - 382421 
 
8th August, 2023  
 
BSE Limited  
P J Towers, 
Dalal Street, 
Mumbai 
{'source': 'Adani Enterprises_aug23.pdf', 'page': 0}


In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

chunk_size =26
chunk_overlap = 4

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [18]:
# Recursive text Splitter
text1 = 'abcdefghijklmnopqrstuvwxyz'
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
print(r_splitter.split_text(text1))

print(r_splitter.split_text(text2))

print(r_splitter.split_text(text3))


# Character Text Splitter
print(c_splitter.split_text(text1))

print(c_splitter.split_text(text2))

print(c_splitter.split_text(text3))


# Character Text Splitter with separator defined
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)

print(c_splitter.split_text(text3))


['abcdefghijklmnopqrstuvwxyz']
['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']
['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']
['abcdefghijklmnopqrstuvwxyz']
['abcdefghijklmnopqrstuvwxyzabcdefg']
['a b c d e f g h i j k l m n o p q r s t u v w x y z']
['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']


In [19]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

print(len(some_text))

c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separators=["\n\n", "\n", " ", ""]
)

496


In [20]:
print(r_splitter.split_text(some_text))

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.", 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']


In [21]:
print(c_splitter.split_text(some_text))

['When writing documents, writers will use document structure to group content. This can convey to the reader, which idea\'s are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also,', 'have a space.and words are separated by space.']


In [22]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

chunks = text_splitter.split_documents(docs)

print(len(docs))
print(len(chunks))

12
43


In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function = len,
    )

chunks1 = text_splitter.split_documents(docs)
print(len(docs))
print(len(chunks1))

12
43


In [24]:
!pip install transformers -q
from transformers import AutoTokenizer

In [25]:

tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-large-v2')

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
  tokenizer,
  chunk_size=1000,
  chunk_overlap=150,
)


chunks2 = text_splitter.split_documents(docs)
print(len(docs))
print(len(chunks2))

12
12


In [26]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader(pdf_files[0]),
    PyPDFLoader(pdf_files[0]),
    PyPDFLoader(pdf_files[1]),
    PyPDFLoader(pdf_files[2])
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function = len,
)

#Create a split of the document using the text splitter
splits = text_splitter.split_documents(docs)

In [28]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma

In [29]:
embeddings1 = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                        model_kwargs={'device': 'cpu'})
embeddings2 = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2",
                                    model_kwargs={'device': 'cpu'})

In [30]:
persist_directory = 'vector_db'

# Create the vector store
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embeddings1,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

226


In [31]:
question = "Is there any question management try to avoid"

docs = vectordb.similarity_search(question,k=3)

# Check the length of the document
len(docs)

# Check the content of the first document
docs[0].page_content

# Persist the database to use it later
vectordb.persist()

In [32]:
question = "How much is Adani Enterprises Limited current CAPEX"

# Similarity search with k = 5
docs = vectordb.similarity_search(question,k=5)

# Check for first two results
print(docs[0])
print(docs[1])

page_content='Adani Enterprises Limited  \nAugust 03 , 2023 \n \n Page 6 of 11 \nairport side , we will have capex this year just about US $ 1.1 billion , all of this just to clarify we \nare assuming  Rs.80 to a dollar rate , adjust for that , so about US $1.1 billion this year would be \nthe CAPEX on airport. It will broadly remain in that range for the next  year then there will be \ndecline , once we complete the first phase  of our development plan in airports . \nModerator : Thank you. The next question is from the line of Nikhil Abhyankar from ICICI Securities. Please \ngo ahead.  \nNikhil Abhyankar : Congrats on a good set of numbers . What is the guidance for commercial mining this year?  And \nthe reason for asking the question is  our production has fallen like 10% Y-o-Y, so what is the \nexact reason for the same?  \nVinay Prakash : As for your first question  about the commercial mines, we are hopeful of starting the open  cut \nin Dhirauli mine which is the commercial min

In [33]:
question = "What is the overall CAPEX plan for the company in FY24"

docs = vectordb.similarity_search(question,k=5)


# Print the metadata of the similarity search result
for doc in docs:
    print(doc.metadata)

print(docs[4].page_content)

{'page': 7, 'source': 'Adani Enterprises_aug23.pdf'}
{'page': 7, 'source': 'Adani Enterprises_aug23.pdf'}
{'page': 7, 'source': 'Adani Enterprises_aug23.pdf'}
{'page': 7, 'source': 'Adani Enterprises_aug23.pdf'}
{'page': 5, 'source': 'Adani Enterprises_Feb23.pdf'}
that ti me, we will come out with the exact numbers also.  
Dhananjay Mishra:  And we have not considered debt of New Bombay airport in this?  
Saurabh Shah : So whatever disbursement have been taken are all included, sir..  
Moderator:   The next question is from the line  of Bajrang Bafna from Sunidhi Securities.  
Bajrang Bafna:  And we genuinely stand by these difficult times, which we are facing due to some unforeseen  
circumstances. So sir, for the benefit of all of us, there is one thing that is in all of our minds that 
how we are going to fund the ongoing capex or the ongoing different projects that we are doing 
maybe for FY '23 and FY '24, which has already been announced. So if you could in a nutshell, 
can defin

In [34]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

smalldb = Chroma.from_texts(texts, embedding=embeddings1)
question = "Tell me about all-white mushrooms with large fruiting bodies"
smalldb.max_marginal_relevance_search(question,k=2, fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

In [35]:
# Compare the result of similarity searcha nd MMR search
question = "what did they say about matlab?"
docs_ss = vectordb.similarity_search(question,k=3)
print(docs_ss[0].page_content[:100])
print(docs_ss[1].page_content[:100])

docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)
print(docs_mmr[0].page_content[:100])
print(docs_mmr[1].page_content[:100])

facility is to control the input cost of green hydrogen, and that's where our focus remains, is to 

facility is to control the input cost of green hydrogen, and that's where our focus remains, is to 

facility is to control the input cost of green hydrogen, and that's where our focus remains, is to 

incubating businesses and our continuing businesses.  
Nikhil Abhyanka r: Understood. And, sir, anyt


In [36]:
question = "Are there any plans to accelerate CAPEX in Adani Enterprise"

docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":pdf_files[0]}
)

# Print metadata of the document retrieved
for d in docs:
    print(d.metadata)

{'page': 7, 'source': 'Adani Enterprises_aug23.pdf'}
{'page': 7, 'source': 'Adani Enterprises_aug23.pdf'}
{'page': 7, 'source': 'Adani Enterprises_aug23.pdf'}


In [37]:
from langchain import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(
    model_id="bigscience/bloomz-1b7",
    task="text-generation",
    model_kwargs={"temperature" : 0, "max_length" : 1000})

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [38]:
doc_retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={'fetch_k': 3})

In [39]:
from langchain.chains import RetrievalQA

earnings_qa = RetrievalQA.from_chain_type(llm=llm,
                                          chain_type="stuff",
                                          retriever=doc_retriever)

In [40]:
earnings_qa.run("How much is the company's capital expenditure?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


' not enough information'

In [41]:
earnings_qa.run("explain in detail how the USD 3.7 billion CAPEX plan for this year will be allocated across different segments.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 1054, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


' Robbie'